## improting libraries


In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Reading dataset

In [20]:
ds = pd.read_csv('../../../../Dataset/Fertilizer Prediction.csv')
ds.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,38,Sandy,Maize,37,0,0,Urea
1,29,52,45,Loamy,Sugarcane,12,0,36,DAP
2,34,65,62,Black,Cotton,7,9,30,14-35-14
3,32,62,34,Red,Tobacco,22,0,20,28-28
4,28,54,46,Clayey,Paddy,35,0,0,Urea


In [21]:
ds.describe()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,30.282828,59.151515,43.181818,18.909091,3.383838,18.606061
std,3.502304,5.840331,11.271568,11.599693,5.814667,13.476978
min,25.000000,50.000000,25.000000,4.000000,0.000000,0.000000
25%,28.000000,54.000000,34.000000,10.000000,0.000000,9.000000
50%,30.000000,60.000000,41.000000,13.000000,0.000000,19.000000
75%,33.000000,64.000000,50.500000,24.000000,7.500000,30.000000
max,38.000000,72.000000,65.000000,42.000000,19.000000,42.000000


In [22]:
ds['Soil Type'].unique()

array(['Sandy', 'Loamy', 'Black', 'Red', 'Clayey'], dtype=object)

In [23]:
ds['Crop Type'].unique()

array(['Maize', 'Sugarcane', 'Cotton', 'Tobacco', 'Paddy', 'Barley',
       'Wheat', 'Millets', 'Oil seeds', 'Pulses', 'Ground Nuts'],
      dtype=object)

## Data Preprocessing

In [24]:
y = ds['Fertilizer Name'].copy()
X = ds.drop('Fertilizer Name', axis=1).copy()

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define feature names for OneHotEncoder
ohe_feature_names = ['Soil Type', 'Crop Type']

# Define ColumnTransformer without specifying feature names
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3, 4])], remainder='passthrough')
X_encoded = ct.fit_transform(X)

# Get feature names after OneHotEncoding
encoded_categories = ct.named_transformers_['encoder'].categories_
encoded_feature_names = []
for i, category in enumerate(encoded_categories):
    for value in category:
        encoded_feature_names.append(f'{ohe_feature_names[i]}_{value}')

# Combine feature names
all_feature_names = encoded_feature_names + ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']

# Reassign feature names to X
X = pd.DataFrame(X_encoded, columns=all_feature_names)


In [28]:
X[0]

KeyError: 0

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=42)



In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [31]:
# Let's print the first row of X_train after scaling
print(X_train[0])

[-0.48181206  1.98206242 -0.59408853 -0.50452498 -0.41169348 -0.24806947
 -0.33601075 -0.33601075 -0.17277369 -0.36214298 -0.24806947 -0.36214298
  3.24037035 -0.43549417 -0.24806947 -0.36214298 -1.50323411 -1.54242294
 -1.00710689  0.46072126 -0.57643157  0.09092764]


## RandomForest

In [32]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42)
classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [33]:
y_pred = classifier.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[2 0 1 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 2 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 6 0 0]
 [0 0 0 0 0 7 0]
 [0 0 0 0 0 0 7]]
0.9666666666666667


In [44]:
import warnings
# Assume you have new data
new_data = np.array([[26, 52, 35, 'Sandy', 'Barley', 12, 10, 13]])

# Apply the same transformations to new_data
new_data_encoded = ct.transform(new_data)
new_data_scaled = sc.transform(new_data_encoded)

# Get feature names after encoding
encoded_feature_names = ct.named_transformers_['encoder'].get_feature_names_out(input_features=ohe_feature_names)
# Combine feature names
all_feature_names_new = list(encoded_feature_names) + ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']

# Make Prediction
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    prediction = classifier.predict(new_data_scaled)

print(prediction[0])

17-17-17


c:\Users\MIS\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\MIS\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
import joblib

# Save the model to a file
joblib.dump(classifier, '../models/FertRecModel.joblib')

['../models/FertRecModel.joblib']

In [21]:
import joblib
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

loaded_model = joblib.load('../models/FertRecModel.joblib')
new_data = np.array([[28, 54, 37, 'Black', 'Millets', 36, 0, 0]])

new_data = ct.transform(new_data)

new_data = sc.transform(new_data)

# Make Prediction
prediction = loaded_model.predict(new_data)

print("Predicted Fertilizer Name:", prediction)


## Gaussian Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2 1 0 0 0 0 0]
 [1 2 1 0 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 5 0 0]
 [1 1 0 2 0 3 0]
 [0 0 0 5 1 0 1]]


0.4666666666666667

## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25)
knn.fit(X,y)

y_pred = knn.predict(X)
cm = confusion_matrix(y, y_pred)
print(cm)
accuracy_score(y, y_pred)


[[ 2  3  0  2  0  0  0]
 [ 0 10  0  0  0  4  0]
 [ 0  1  0  5  1  0  0]
 [ 0  0  0  8  6  0  0]
 [ 0  0  0  0 17  0  0]
 [ 0  0  0  0  0 18  0]
 [ 0  0  0  0  0  0 22]]


0.7777777777777778